In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install kaggle
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !kaggle datasets download -d watchman/rtsd-dataset
# !unzip rtsd-dataset.zip
# !rm rtsd-dataset.zip
# !cp -r rtsd-frames/rtsd-frames/ .
# !rm -r rtsd-frames/rtsd-frames/

In [ ]:
import numpy as np
import pandas as pd
import os
import json
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
import seaborn as sns
import random
import cv2
from pycocotools.coco import COCO
import time

import warnings
warnings.filterwarnings('ignore')

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from sklearn import metrics

import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from torchvision.models import mobilenet_v3_large, resnet152
import torch.nn as nn
from torch.nn import functional as F

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

### CLF_mobilenet_v3_large на большой выборке train_anno + Аугментация данных + class не знак

In [ ]:
device_id = 0
device = 'cpu' if device_id == -1 else f'cuda:{device_id}'

n_epochs = 10
batch_size = 100
classes = 2
classes1 = 157
device

'cuda:0'

In [ ]:
# функция загрузки датасета в pd
def dataset(path, name_f):
  
  with open(os.path.join(path, name_f), 'r') as f:
    anno = json.load(f)

    obj1 = anno.get('images')
    df1 = pd.json_normalize(obj1)
    obj2 = anno.get('annotations')
    df2 = pd.json_normalize(obj2)
    obj3 = anno.get('categories')
    df3 = pd.json_normalize(obj3)
    t = df2.merge(df3.set_index('id'), left_on='category_id',right_index=True)
    df= t.merge(df1.set_index('id'), left_on='image_id',right_index=True)
  
  return df

In [ ]:
df_train_anno = dataset('.', 'train_anno.json')
df_val_anno = dataset('.', 'val_anno.json')
df_train_anno_reduced = dataset('.', 'train_anno_reduced.json')

In [ ]:
%cd ..

/content


In [ ]:
dfa = df_train_anno.copy().reset_index()
dfa = dfa.drop(['index'],axis=1)
dfa.iloc[10:100, 4] = dfa.iloc[10:100, 4].apply(lambda x: [int(x[0]-0.6*x[2]), int(x[1]-0.6*x[3]), x[2], x[3]])
dfa.iloc[10:100, 2] = 156
dfa.iloc[10:100, 6] = '0_0'
dfa

,id,image_id,category_id,area,bbox,iscrowd,name,width,height,file_name
0,0,0,1,324,"[649, 376, 18, 18]",0,2_1,1280,720,rtsd-frames/autosave01_02_2012_09_13_33.jpg
1,1,1,1,420,"[671, 356, 20, 21]",0,2_1,1280,720,rtsd-frames/autosave01_02_2012_09_13_34.jpg
2,2,2,1,702,"[711, 332, 27, 26]",0,2_1,1280,720,rtsd-frames/autosave01_02_2012_09_13_35.jpg
3,3,3,1,1332,"[764, 290, 37, 36]",0,2_1,1280,720,rtsd-frames/autosave01_02_2012_09_13_36.jpg
4,4,3,2,289,"[684, 384, 17, 17]",0,1_23,1280,720,rtsd-frames/autosave01_02_2012_09_13_36.jpg
...,...,...,...,...,...,...,...,...,...,...
95487,70506,38149,149,2100,"[1007, 307, 35, 60]",0,6_8_1,1280,720,rtsd-frames/autosave16_10_2012_08_36_42_2.jpg
95488,70507,38150,149,3150,"[1088, 296, 45, 70]",0,6_8_1,1280,720,rtsd-frames/autosave16_10_2012_08_36_43_0.jpg
95489,102242,57700,154,1519,"[712, 329, 31, 49]",0,7_14,1280,720,rtsd-frames/autosave24_10_2013_11_23_19_1.jpg
95490,102243,57701,154,2340,"[767, 306, 39, 60]",0,7_14,1280,720,rtsd-frames/autosave24_10_2013_11_23_19_2.jpg


In [ ]:
class RTSD_dataset_clf_my(Dataset):
    def __init__(self, path_img, df, transforms=None):
        super().__init__()
        self.path_img = path_img
        self.transforms = transforms
        self.df = df
        
    def __getitem__(self, index):
        name_img = self.df.loc[index,'file_name']
        boxes = self.df.loc[index,'bbox']
        bb = [boxes[0], boxes[1], boxes[0] + boxes[2], boxes[1] + boxes[3]]
        imgs = Image.open(os.path.join(self.path_img, name_img))
        imgs = imgs.crop(bb)

        if self.transforms is not None:
            imgs = self.transforms(imgs)

        imgs = imgs / 255

        targets = torch.tensor(self.df.loc[index, 'category_id'])


#         return imgs, targets
        return {
                'images': imgs,
                'targets': targets}
    
    def __len__(self):
        
        return self.df.shape[0]

In [ ]:
def get_transform1():
#     custom_transforms = []
#     custom_transforms.append(transforms.ToTensor())
    return transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

In [ ]:
def get_transform_a1():
             return transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomChoice([transforms.Compose([transforms.RandomPerspective(distortion_scale=0.4,p=0.9),
                                                                                    transforms.ToTensor()
                                                                                    ]),
                                                                transforms.Compose([transforms.ColorJitter(brightness=(0.4), contrast=(0.3), saturation=(0.3)),
                                                                                    transforms.ToTensor()
                                                                                    ]),
                                                                transforms.Compose([transforms.RandomResizedCrop((224,224), scale=(0.85, 1)), # Случайная обрезка изображения в диапахоне 85 - 100% и resize в исходный размер
                                                                                    transforms.ToTensor()
                                                                                    ])        
                                                                ])
                                       ])


In [ ]:
df222 = RTSD_dataset_clf_my('./', dfa, transforms=get_transform_a1())
df122 = RTSD_dataset_clf_my('./', df_val_anno, transforms=get_transform1())

In [ ]:
data_loader_train22 = torch.utils.data.DataLoader(df222,
                                          batch_size=batch_size,
                                          shuffle=True, drop_last = True)

data_loader_val22 = torch.utils.data.DataLoader(df122,
                                          batch_size=batch_size,
                                          shuffle=False, drop_last = True)

In [ ]:
def build_model_clf(classes):
    model = mobilenet_v3_large(weights='MobileNet_V3_Large_Weights.IMAGENET1K_V2')
    for param in model.parameters():
        param.requires_grad = False
    model.classifier[3] = nn.Linear(in_features=model.classifier[3].in_features, out_features=classes)
#     model.classifier[3] = nn.Linear(in_features=1280, out_features=classes)
    for param in  model.classifier.parameters():
        param.requires_grad = True

    return model

In [ ]:
#model_clf_11 
model = build_model_clf(classes1).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.001)
loss_func = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 93.8MB/s]


In [ ]:
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []   

y_true_t, y_true_v = [], []
y_pred_t, y_pred_v = [], []

for epoch in range(n_epochs):
    start = time.time()
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    train_class_correct = list(0. for i in range(classes1))
    train_class_total = list(0. for i in range(classes1))
    for i, batch in enumerate(data_loader_train22):
        optimizer.zero_grad()
        image, targets = batch['images'].to(device), batch['targets'].to(device)
#         image = list(img.to(device) for img in image)
#         targets = [t.to(device) for t in targets]
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        loss = loss_func(outputs, targets)
        train_running_loss += loss.item()
        # Calculate the accuracy.
        _, preds = torch.max(outputs, 1)
        train_running_correct += (preds == targets).sum().item()
        correct  = (preds == targets).squeeze()
        for a in range(len(preds)):
                label = targets[a]
                # print(label)
                train_class_correct[label] += correct[a].item()
                train_class_total[label] += 1
        # Backpropagation.
        loss.backward()
        # Update the weights.
        optimizer.step()     
        
        y_true_t.extend([int(item) for item in targets])
        y_pred_t.extend([int(item) for item in preds])
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    
    # Loss and accuracy for the complete epoch.
    train_epoch_loss = train_running_loss / i
    train_epoch_acc = 100. * (train_running_correct / len(df222))
    
    
    
    train_epoch_accuracy = metrics.accuracy_score(y_true_t, y_pred_t)
    train_epoch_f1_micro = metrics.f1_score(y_true_t, y_pred_t, average="micro")
    train_epoch_f1_macro =  metrics.f1_score(y_true_t, y_pred_t, average="macro")
    train_epoch_f1_weighted = metrics.f1_score(y_true_t, y_pred_t, average="weighted")
    
    
    
    print('Эпоха train:', epoch,'Итераций:', i, 'train loss:', train_epoch_loss, 'train acc:', train_epoch_acc)
    
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    # We need two lists to keep track of class-wise accuracy.
    class_correct = list(0. for i in range(classes1))
    class_total = list(0. for i in range(classes1))
    with torch.no_grad():
        for i, batch in enumerate(data_loader_val22):
#             image, targets = batch
            image, targets = batch['images'].to(device), batch['targets'].to(device)
#             image = list(img.to(device) for img in image)
#             targets = [t.to(device) for t in targets]
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = loss_func(outputs, targets)
            valid_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs, 1)
            valid_running_correct += (preds == targets).sum().item()
            # Calculate the accuracy for each class.
            correct  = (preds == targets).squeeze()
            for j in range(len(preds)):
                label = targets[j]
                # print(label)
                class_correct[label] += correct[j].item()
                class_total[label] += 1
        
            if i % 200 == 0:
                 print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
                    
            y_true_v.extend([int(item) for item in targets])
            y_pred_v.extend([int(item) for item in preds])
        
    # Loss and accuracy for the complete epoch.
    valid_epoch_loss = valid_running_loss / i
    valid_epoch_acc = 100. * (valid_running_correct / len(df122))
    
    valid_epoch_accuracy = metrics.accuracy_score(y_true_v, y_pred_v)
    valid_epoch_f1_micro = metrics.f1_score(y_true_v, y_pred_v, average="micro")
    valid_epoch_f1_macro =  metrics.f1_score(y_true_v, y_pred_v, average="macro")
    valid_epoch_f1_weighted = metrics.f1_score(y_true_v, y_pred_v, average="weighted")
    
    end = time.time()
    print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
    print('Эпоха val:', epoch, 'Итераций:', i, 'val loss:', valid_epoch_loss, 'val acc:', valid_epoch_acc)
    
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    
    torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'accuracy_train': train_epoch_accuracy,
                    'f1_micro_trainl': train_epoch_f1_micro,
                    'f1_macro_train': train_epoch_f1_macro,
                    'f1_weighted_train': train_epoch_f1_weighted,
                    'loss_train': train_loss,
                    'acc_train': train_acc,
                    'accuracy_val': valid_epoch_accuracy,
                    'f1_micro_val': valid_epoch_f1_micro,
                    'f1_macro_val': valid_epoch_f1_macro,
                    'f1_weighted_val': valid_epoch_f1_weighted,
                    'loss_val': valid_loss,
                    'acc_val': valid_acc
                    }, f'./chkpt_modelv3a_clf_{epoch}.pth')
    
    torch.save(model.state_dict(), f'./chkpt_mv3a_clf_{epoch}.pth')
    !cp chkpt_mv3a_clf_{epoch}.pth /content/drive/MyDrive
    !cp chkpt_modelv3a_clf_{epoch}.pth /content/drive/MyDrive

epoch: 0, step: 0, loss: 5.026838302612305
epoch: 0, step: 200, loss: 1.0423680543899536
epoch: 0, step: 400, loss: 1.0384654998779297
epoch: 0, step: 600, loss: 0.7420834898948669
epoch: 0, step: 800, loss: 0.7455179691314697
Эпоха train: 0 Итераций: 953 train loss: 1.0330450365638433 train acc: 73.75382230972228
epoch: 0, step: 0, loss: 0.8517508506774902
Потрачено 31.0 минут на 0 эпоху
Эпоха val: 0 Итераций: 87 val loss: 0.6932352997105697 val acc: 80.90457929167607
epoch: 1, step: 0, loss: 0.6320469379425049
epoch: 1, step: 200, loss: 0.6871518492698669
epoch: 1, step: 400, loss: 0.5698065757751465
epoch: 1, step: 600, loss: 0.8065493702888489
epoch: 1, step: 800, loss: 0.9976919293403625
Эпоха train: 1 Итераций: 953 train loss: 0.6446597889175195 train acc: 81.70317932392243
epoch: 1, step: 0, loss: 0.5056474804878235
Потрачено 31.1 минут на 1 эпоху
Эпоха val: 1 Итераций: 87 val loss: 0.45386368875530947 val acc: 86.5666591473043
epoch: 2, step: 0, loss: 0.5747249126434326
epoch: 

### DETECTOR fasterrcnn_resnet50_fpn_v2 + Аугментация данных

In [ ]:
device_id = 0
device = 'cpu' if device_id == -1 else f'cuda:{device_id}'

n_epochs = 10
batch_size = 10
classes = 2
classes1 = 157
device

'cuda:0'

In [ ]:
# Приведем все знаки к одному классу
df_train_anno_1 = df_train_anno.copy()
df_train_anno_1['category_id'] = 1

df_val_anno_1 = df_val_anno.copy()
df_val_anno_1['category_id'] = 1

df_train_anno_reduced_1 = df_train_anno_reduced.copy()
df_train_anno_reduced_1['category_id'] = 1

In [ ]:
class RTSD_dataset_my_a(Dataset):
    def __init__(self, path_img, df, transforms=None):
        super().__init__()
        self.path_img = path_img
        self.transforms = transforms
        self.df = df
        self.df = self.df.groupby('file_name', as_index=False).agg(list)
        
    def __getitem__(self, index):
        name_img = self.df.loc[index,'file_name']
        img = Image.open(os.path.join(self.path_img, name_img))
        img = np.array(img)
      
        boxes = []
        for b in self.df.loc[index,'bbox']:
            bb = [b[0], b[1], b[0] + b[2], b[1] + b[3]]
            boxes.append(bb)   

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.Tensor(self.df.loc[index, 'category_id']).to(torch.int64)
        img_id = torch.tensor([self.df.loc[index, 'image_id'][0]])
        
        areas = []
        for i in self.df.loc[index,'area']:
            areas.append(self.df.loc[index,'area'])
        areas = torch.as_tensor(areas[0], dtype=torch.float32)
       
        iscrowd = torch.zeros(len(self.df.loc[index,'iscrowd'])).to(torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

#         if self.transforms is not None:
#             img = self.transforms(img)

        if self.transforms:
                    sample = self.transforms(image = img,
                                             bboxes =  my_annotation['boxes'],
                                             labels = labels)
                    img = sample['image']
                    my_annotation['boxes'] = torch.Tensor(sample['bboxes'])
        
        img = img / 255

        return img, my_annotation

    def __len__(self):
        
        return self.df.shape[0]

In [ ]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# define the training tranforms
def get_train_transform():
    return A.Compose([
        A.MotionBlur(blur_limit=3, p=0.2),
        A.Blur(blur_limit=3, p=0.1),
        A.RandomBrightnessContrast(
            brightness_limit=0.2, p=0.5
        ),
        A.ColorJitter(p=0.5),
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })
# define the validation transforms
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })

In [ ]:
df2 = RTSD_dataset_my_a('./', df_train_anno_1,transforms=get_train_transform())
df22 = RTSD_dataset_my_a('./', df_val_anno_1,transforms=get_valid_transform())

In [ ]:
data_loader_train2 = torch.utils.data.DataLoader(df2,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          collate_fn=collate_fn,
                                          drop_last = True)

data_loader_val2 = torch.utils.data.DataLoader(df22,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          collate_fn=collate_fn,
                                          drop_last = True)


In [ ]:
def build_model(n_model, classes):
    if 'resnet50_fpn_v2' in n_model:
        model =torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights='FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1')
    elif 'resnet50_fpn' in n_model:
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="FasterRCNN_ResNet50_FPN_Weights.COCO_V1")
    else:
        print("нет такой модели ")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, classes)
    
    return model

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath, map_location=device)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['model_state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['loss_val']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

model = build_model('resnet50_fpn_v2', 2).to(device)
params1 = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params1, lr=0.001)

model1, optimizer1, start_epoch, loss1_val = load_ckp('./chkpt_model2_d_6.pth', model, optimizer)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:01<00:00, 135MB/s]


In [ ]:
model1 = build_model('resnet50_fpn_v2', classes).to(device)
params1 = [p for p in model1.parameters() if p.requires_grad]
optimizer1 = torch.optim.Adam(params1, lr=0.001)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:01<00:00, 147MB/s]


In [ ]:
loss1_train = []
loss1_val = []

for epoch in range(7, n_epochs):
    start = time.time()
    model1.train()
    for i, batch in enumerate(data_loader_train2):
        optimizer1.zero_grad()
        loss_train = 0
        imgs, targets = batch
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model1(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_train += losses.item()
        losses.backward()
        optimizer1.step()
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {losses.item()}')
    
    loss11_train = (loss_train / i)
    loss1_train.append(loss11_train)
    
    print('Эпоха train:', epoch,'Итераций:', i, 'train loss:', (loss_train / i))
    
    for i, batch in enumerate(data_loader_val2):
        optimizer1.zero_grad()
        loss_val = 0
        imgs, targets = batch
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
              loss_dict = model1(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_val += losses.item()
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {losses.item()}')
    
    loss11_val = (loss_val / i)
    loss1_val.append(loss11_val)

    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=3, gamma=0.9)

    end = time.time()
    print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
    print('Эпоха val:', epoch, 'Итераций:', i, 'val loss:', (loss_val / i))
    
        
    torch.save({'epoch': epoch,
                    'model_state_dict': model1.state_dict(),
                    'optimizer_state_dict': optimizer1.state_dict(),
                    'lr_scheduler_state_dict': lr_scheduler.state_dict(),
                    'loss_train': loss1_train,
                    'loss_val': loss1_val,
                    }, f'./chkpt_model2_d_{epoch}.pth')
    
    torch.save(model1.state_dict(), f'./chkpt_m2_d_{epoch}.pth')
    !cp chkpt_model2_d_{epoch}.pth /content/drive/MyDrive
    !cp chkpt_m2_d_{epoch}.pth /content/drive/MyDrive

epoch: 7, step: 0, loss: 0.0753016471862793
epoch: 7, step: 200, loss: 0.04973529651761055
epoch: 7, step: 400, loss: 0.10200314968824387
epoch: 7, step: 600, loss: 0.1114787831902504
epoch: 7, step: 800, loss: 0.06412721425294876
epoch: 7, step: 1000, loss: 0.06871630251407623
epoch: 7, step: 1200, loss: 0.05390813201665878
epoch: 7, step: 1400, loss: 0.06693138927221298
epoch: 7, step: 1600, loss: 0.07883910834789276
epoch: 7, step: 1800, loss: 0.07663627713918686
epoch: 7, step: 2000, loss: 0.048805687576532364
epoch: 7, step: 2200, loss: 0.03328828141093254
epoch: 7, step: 2400, loss: 0.049430664628744125
epoch: 7, step: 2600, loss: 0.07334597408771515
epoch: 7, step: 2800, loss: 0.07179352641105652
epoch: 7, step: 3000, loss: 0.0659819170832634
epoch: 7, step: 3200, loss: 0.05928252637386322
epoch: 7, step: 3400, loss: 0.08463607728481293
epoch: 7, step: 3600, loss: 0.06910812109708786
epoch: 7, step: 3800, loss: 0.06959029287099838
epoch: 7, step: 4000, loss: 0.10676819086074829


In [ ]:
model1 = build_model('resnet50_fpn', 2).to(device)
params1 = [p for p in model1.parameters() if p.requires_grad]
optimizer1 = torch.optim.Adam(params1, lr=0.001)

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath, map_location=device)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['model_state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['loss_val']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

model = build_model('resnet50_fpn', 2).to(device)
params1 = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params1, lr=0.001)

model1, optimizer1, start_epoch, loss1_val = load_ckp('./chkpt_model1aa_d_0.pth', model, optimizer)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 333MB/s]


In [71]:
loss1_train = []
loss1_val = []

for epoch in range(1, n_epochs):
    start = time.time()
    model1.train()
    for i, batch in enumerate(data_loader_train2):
        optimizer1.zero_grad()
        loss_train = 0
        imgs, targets = batch
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model1(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_train += losses.item()
        losses.backward()
        optimizer1.step()
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {losses.item()}')
    
    loss11_train = (loss_train / i)
    loss1_train.append(loss11_train)
    
    print('Эпоха train:', epoch,'Итераций:', i, 'train loss:', (loss_train / i))
    
    for i, batch in enumerate(data_loader_val2):
        optimizer1.zero_grad()
        loss_val = 0
        imgs, targets = batch
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
              loss_dict = model1(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_val += losses.item()
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {losses.item()}')
    
    loss11_val = (loss_val / i)
    loss1_val.append(loss11_val)
    
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=3, gamma=0.9)
    
    end = time.time()
    print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
    print('Эпоха val:', epoch, 'Итераций:', i, 'val loss:', (loss_val / i))
    
        
    torch.save({'epoch': epoch,
                    'model_state_dict': model1.state_dict(),
                    'optimizer_state_dict': optimizer1.state_dict(),
                    'lr_scheduler_state_dict': lr_scheduler.state_dict(),
                    'loss_train': loss1_train,
                    'loss_val': loss1_val,
                    }, f'./chkpt_model1aa_d_{epoch}.pth')
    
    torch.save(model1.state_dict(), f'./chkpt_m1aa_d_{epoch}.pth')
    !cp chkpt_model1aa_d_{epoch}.pth /content/drive/MyDrive
    !cp chkpt_m1aa_d_{epoch}.pth /content/drive/MyDrive

epoch: 1, step: 0, loss: 0.07571166008710861
epoch: 1, step: 200, loss: 0.07042182981967926
epoch: 1, step: 400, loss: 0.11172984540462494
epoch: 1, step: 600, loss: 0.10133524984121323
epoch: 1, step: 800, loss: 0.11728022992610931
epoch: 1, step: 1000, loss: 0.14196282625198364
epoch: 1, step: 1200, loss: 0.08096003532409668
epoch: 1, step: 1400, loss: 0.08901054412126541
epoch: 1, step: 1600, loss: 0.10958066582679749
epoch: 1, step: 1800, loss: 0.133158877491951
epoch: 1, step: 2000, loss: 0.16329233348369598
epoch: 1, step: 2200, loss: 0.08531792461872101
epoch: 1, step: 2400, loss: 0.07471255958080292
epoch: 1, step: 2600, loss: 0.08446311205625534
epoch: 1, step: 2800, loss: 0.06701342761516571
epoch: 1, step: 3000, loss: 0.0834004357457161
epoch: 1, step: 3200, loss: 0.1365889459848404
epoch: 1, step: 3400, loss: 0.8244222402572632
epoch: 1, step: 3600, loss: 0.17560029029846191
epoch: 1, step: 3800, loss: 0.5143008232116699
epoch: 1, step: 4000, loss: 0.6085484623908997
epoch:

### CLF_mobilenet_v3_large на маленькой выборке train_anno_reduced увеличенной в 20 раз + Аугментация данных  на большой выборке train_anno + Аугментация данных + class не знак

In [72]:
device_id = 0
device = 'cpu' if device_id == -1 else f'cuda:{device_id}'

n_epochs = 100
batch_size = 100
classes = 2
classes1 = 157
device

'cuda:0'

In [73]:
def dataset(path, name_f):
  
  with open(os.path.join(path, name_f), 'r') as f:
    anno = json.load(f)

    obj1 = anno.get('images')
    df1 = pd.json_normalize(obj1)
    obj2 = anno.get('annotations')
    df2 = pd.json_normalize(obj2)
    obj3 = anno.get('categories')
    df3 = pd.json_normalize(obj3)
    t = df2.merge(df3.set_index('id'), left_on='category_id',right_index=True)
    df= t.merge(df1.set_index('id'), left_on='image_id',right_index=True)
  
  return df

In [74]:
df_train_anno = dataset('.', 'train_anno.json')
df_val_anno = dataset('.', 'val_anno.json')
df_train_anno_reduced = dataset('.', 'train_anno_reduced.json')

In [75]:
class RTSD_dataset_clf_my(Dataset):
    def __init__(self, path_img, df, transforms=None):
        super().__init__()
        self.path_img = path_img
        self.transforms = transforms
        self.df = df
        
    def __getitem__(self, index):
        name_img = self.df.loc[index,'file_name']
        boxes = self.df.loc[index,'bbox']
        bb = [boxes[0], boxes[1], boxes[0] + boxes[2], boxes[1] + boxes[3]]
        imgs = Image.open(os.path.join(self.path_img, name_img))
        imgs = imgs.crop(bb)

        if self.transforms is not None:
            imgs = self.transforms(imgs)

        imgs = imgs / 255

        targets = torch.tensor(self.df.loc[index, 'category_id'])


#         return imgs, targets
        return {
                'images': imgs,
                'targets': targets}
    
    def __len__(self):
        
        return self.df.shape[0]

In [76]:
def get_transform1():
#     custom_transforms = []
#     custom_transforms.append(transforms.ToTensor())
    return transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

In [77]:
def get_transform_a1():
             return transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomChoice([transforms.Compose([transforms.RandomPerspective(distortion_scale=0.4,p=0.9),
                                                                                    transforms.ToTensor()
                                                                                    ]),
                                                                transforms.Compose([transforms.ColorJitter(brightness=(0.4), contrast=(0.3), saturation=(0.3)),
                                                                                    transforms.ToTensor()
                                                                                    ]),
                                                                transforms.Compose([transforms.RandomResizedCrop((224,224), scale=(0.85, 1)), # Случайная обрезка изображения в диапахоне 85 - 100% и resize в исходный размер
                                                                                    transforms.ToTensor()
                                                                                    ])        
                                                                ])
                                       ])


In [78]:
dfb = df_train_anno_reduced.copy().reset_index()
dfb = dfb.drop(['index'],axis=1)
sorted = dfb.sort_values(['category_id'])
first = sorted.groupby('category_id').first().reset_index()

In [79]:
dfa = first.copy().reset_index()
dfa = dfa.drop(['index'],axis=1)
dfa['bbox'] = dfa['bbox'].apply(lambda x: [int(x[0]-0.6*x[2]), int(x[1]-0.6*x[3]), x[2], x[3]])
dfa['category_id'] = 156
dfa['name'] = '0_0'
dfaa = pd.DataFrame(np.repeat(dfa.values, 2, axis=0), columns=dfa.columns)
new_df = pd.DataFrame(np.repeat(df_train_anno_reduced.values, 21, axis=0), columns=df_train_anno_reduced.columns)
frames = [new_df, dfaa]
result = pd.concat(frames).reset_index()
result = result.drop(['index'],axis=1)

In [80]:
dfa = first.copy().reset_index()
dfa = dfa.drop(['index'],axis=1)
dfa['bbox'] = dfa['bbox'].apply(lambda x: [int(x[0]-0.6*x[2]), int(x[1]-0.6*x[3]), x[2], x[3]])
dfa['category_id'] = 156
dfa['name'] = '0_0'
dfaa = pd.DataFrame(np.repeat(dfa.values, 2, axis=0), columns=dfa.columns)
new_df = pd.DataFrame(np.repeat(df_train_anno_reduced.values, 21, axis=0), columns=df_train_anno_reduced.columns)
frames = [new_df, dfaa]
result = pd.concat(frames).reset_index()
result = result.drop(['index'],axis=1)

In [81]:
df12 = RTSD_dataset_clf_my('./', result, transforms=get_transform_a1())

df120 = RTSD_dataset_clf_my('./', df_train_anno_reduced, transforms=get_transform1())

In [82]:
data_loader_train12 = torch.utils.data.DataLoader(df12,
                                          batch_size=batch_size,
                                          shuffle=True, drop_last = True)

data_loader_val12 = torch.utils.data.DataLoader(df120,
                                          batch_size=batch_size,
                                          shuffle=False, drop_last = True)

In [83]:
def build_model_clf(classes):
    model = mobilenet_v3_large(weights='MobileNet_V3_Large_Weights.IMAGENET1K_V2')
    for param in model.parameters():
        param.requires_grad = False
    model.classifier[3] = nn.Linear(in_features=model.classifier[3].in_features, out_features=classes)
#     model.classifier[3] = nn.Linear(in_features=1280, out_features=classes)
    for param in  model.classifier.parameters():
        param.requires_grad = True

    return model

In [84]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath, map_location=device)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['model_state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['loss_val']
    train_loss_min = checkpoint['loss_train']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min, train_loss_min


model1 = build_model_clf(157).to(device)
params1 = [p for p in model1.parameters() if p.requires_grad]
optimizer1 = torch.optim.Adam(params1, lr=0.001)
loss_func = nn.CrossEntropyLoss()

model, optimizer, start_epoch, valid_loss, train_loss = load_ckp('./chkpt_modelrv3_clf_32.pth', model1, optimizer1)

In [ ]:
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []   

y_true_t, y_true_v = [], []
y_pred_t, y_pred_v = [], []

for epoch in range(33, n_epochs):
    start = time.time()
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    train_class_correct = list(0. for i in range(classes1))
    train_class_total = list(0. for i in range(classes1))
    for i, batch in enumerate(data_loader_train12):
        optimizer.zero_grad()
        image, targets = batch['images'].to(device), batch['targets'].to(device)
#         image = list(img.to(device) for img in image)
#         targets = [t.to(device) for t in targets]
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        loss = loss_func(outputs, targets)
        train_running_loss += loss.item()
        # Calculate the accuracy.
        _, preds = torch.max(outputs, 1)
        train_running_correct += (preds == targets).sum().item()
        correct  = (preds == targets).squeeze()
        for a in range(len(preds)):
                label = targets[a]
                # print(label)
                train_class_correct[label] += correct[a].item()
                train_class_total[label] += 1
        # Backpropagation.
        loss.backward()
        # Update the weights.
        optimizer.step()     
        
        y_true_t.extend([int(item) for item in targets])
        y_pred_t.extend([int(item) for item in preds])
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    
    # Loss and accuracy for the complete epoch.
    train_epoch_loss = train_running_loss / i
    train_epoch_acc = 100. * (train_running_correct / len(df12))
    
    
    
    train_epoch_accuracy = metrics.accuracy_score(y_true_t, y_pred_t)
    train_epoch_f1_micro = metrics.f1_score(y_true_t, y_pred_t, average="micro")
    train_epoch_f1_macro =  metrics.f1_score(y_true_t, y_pred_t, average="macro")
    train_epoch_f1_weighted = metrics.f1_score(y_true_t, y_pred_t, average="weighted")
    
    
    
    print('Эпоха train:', epoch,'Итераций:', i, 'train loss:', train_epoch_loss, 'train acc:', train_epoch_acc)
    
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    # We need two lists to keep track of class-wise accuracy.
    class_correct = list(0. for i in range(classes1))
    class_total = list(0. for i in range(classes1))
    with torch.no_grad():
        for i, batch in enumerate(data_loader_val12):
#             image, targets = batch
            image, targets = batch['images'].to(device), batch['targets'].to(device)
#             image = list(img.to(device) for img in image)
#             targets = [t.to(device) for t in targets]
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = loss_func(outputs, targets)
            valid_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs, 1)
            valid_running_correct += (preds == targets).sum().item()
            # Calculate the accuracy for each class.
            correct  = (preds == targets).squeeze()
            for j in range(len(preds)):
                label = targets[j]
                # print(label)
                class_correct[label] += correct[j].item()
                class_total[label] += 1
        
            if i % 200 == 0:
                 print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
                    
            y_true_v.extend([int(item) for item in targets])
            y_pred_v.extend([int(item) for item in preds])
        
    # Loss and accuracy for the complete epoch.
    valid_epoch_loss = valid_running_loss / i
    valid_epoch_acc = 100. * (valid_running_correct / len(df120))
    
    valid_epoch_accuracy = metrics.accuracy_score(y_true_v, y_pred_v)
    valid_epoch_f1_micro = metrics.f1_score(y_true_v, y_pred_v, average="micro")
    valid_epoch_f1_macro =  metrics.f1_score(y_true_v, y_pred_v, average="macro")
    valid_epoch_f1_weighted = metrics.f1_score(y_true_v, y_pred_v, average="weighted")
    
    end = time.time()
    print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
    print('Эпоха val:', epoch, 'Итераций:', i, 'val loss:', valid_epoch_loss, 'val acc:', valid_epoch_acc)
    
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    
    torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'accuracy_train': train_epoch_accuracy,
                    'f1_micro_trainl': train_epoch_f1_micro,
                    'f1_macro_train': train_epoch_f1_macro,
                    'f1_weighted_train': train_epoch_f1_weighted,
                    'loss_train': train_loss,
                    'acc_train': train_acc,
                    'accuracy_val': valid_epoch_accuracy,
                    'f1_micro_val': valid_epoch_f1_micro,
                    'f1_macro_val': valid_epoch_f1_macro,
                    'f1_weighted_val': valid_epoch_f1_weighted,
                    'loss_val': valid_loss,
                    'acc_val': valid_acc
                    }, f'./chkpt_modelrv3_clf_{epoch}.pth')
    
    torch.save(model.state_dict(), f'./chkpt_mrv3_clf_{epoch}.pth')
    !cp chkpt_mrv3_clf_{epoch}.pth /content/drive/MyDrive
    !cp chkpt_modelrv3_clf_{epoch}.pth /content/drive/MyDrive

epoch: 33, step: 0, loss: 0.006391296163201332
epoch: 33, step: 200, loss: 0.004785778932273388
epoch: 33, step: 400, loss: 0.03172260895371437
Эпоха train: 33 Итераций: 490 train loss: 0.055775482388811984 train acc: 98.3535158237509
epoch: 33, step: 0, loss: 2.4102059796859976e-06
Потрачено 14.5 минут на 33 эпоху
Эпоха val: 33 Итераций: 22 val loss: 5.125233752653361e-06 val acc: 98.9247311827957
epoch: 34, step: 0, loss: 0.022283880040049553
epoch: 34, step: 200, loss: 0.031919412314891815
epoch: 34, step: 400, loss: 0.09679806977510452
Эпоха train: 34 Итераций: 490 train loss: 0.05802603298424011 train acc: 98.25379057698179
epoch: 34, step: 0, loss: 3.217119683540659e-06
Потрачено 14.5 минут на 34 эпоху
Эпоха val: 34 Итераций: 22 val loss: 1.0774189835345686e-05 val acc: 98.9247311827957
epoch: 35, step: 0, loss: 0.04898535832762718
epoch: 35, step: 200, loss: 0.12349717319011688
epoch: 35, step: 400, loss: 0.00483278650790453
Эпоха train: 35 Итераций: 490 train loss: 0.0577985548